In [10]:

from sklearnex import patch_sklearn, config_context
patch_sklearn()

import warnings
warnings.filterwarnings("error")

#approxximating
from sklearn.kernel_approximation import Nystroem

from sklearn import preprocessing
import pandas as pd
import sklearn.svm as svm
from sklearn.ensemble import BaggingClassifier
from sklearn import tree
from scipy.io import arff
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
#from category_encoders import *

import time
from sklearn.preprocessing import MinMaxScaler
from abc import ABC, abstractmethod
from sklearn import tree
from math import trunc
from math import ceil
from sklearn.neighbors import KNeighborsClassifier
#from c45 import C45
from sklearn.model_selection import train_test_split, cross_val_score
import matplotlib.pyplot as plt
#from tabulate import tabulate

#sklearnex
#import sys
#sys.path.insert(0, "C:\Users\bapti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearnex")
#import 
#import daal4py.sklearn


#regex
import re

#This is for optimization

#For randomness:
import random

#For different classifiers:
from sklearn.neural_network import MLPClassifier
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis



Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [11]:



clf = svm.SVC(kernel="linear", C=1)
clf2= svm.SVC(kernel="linear", C=1)
pipe = make_pipeline(StandardScaler(),clf)
X = np.array([[-1, -1], [-2, -1],[-3,-1],[3,1], [1, 1], [2, 1],[2.5,1],[1.5,1]])
Z = np.array([[-1, -1], [-2, -1], [1, 1], [2, 1]])
A = np.array([1, 1, 2,2])
Y = np.array([1, 1, 2, 1, 2, 1,2,2])
clf.fit(X,Y)
clf2.fit(Z,A)
R=np.array_split(X,3)
#print(R)
#ARF load, py.io
# Target, and ordinal is good.
# anaconda and jupyter notebook

# svm.Pipeline(steps=[('standardscaler', StandardScaler()),
#                 ('svc', SVC(gamma='auto'))])

#print(clf.predict([[-.8,-1]]))
vecotr= clf.support_vectors_
vecotr2= clf2.support_vectors_
#print(clf.decision_function(vecotr))
#print(clf2.decision_function(vecotr2))
marginDensity= 0


#polymorphism vs the abstract factory. Make the code well.Why not an abstract factor. cuz drift detector with the thing. needs it.

class MDDriftDetector(ABC):

    def KXVal(self, data, labels,K,*classifierSettings):
        splitData=np.array_split(data,K)
        splitLabels=np.array_split(labels,K)
        accuracies = np.zeros(K)
        densities = np.zeros(K)
        for i in range(K):
            trainData = np.concatenate(splitData[:i]+splitData[i+1:])
            trainLabels= np.concatenate(splitLabels[:i]+splitLabels[i+1:])
            dens,acc= self.CrossValid(trainData,splitData[i],trainLabels,splitLabels[i],classifierSettings)
            accuracies[i]= acc
            densities[i] = dens
        averageAccuracy= np.average(accuracies)
        standardDevAccuracy= np.std(accuracies)
        averageDensity=np.average(densities)
        standardDevDensity=np.std(densities)
        return averageAccuracy, standardDevAccuracy,averageDensity,standardDevDensity

    def CrossValid(self,trainData,testData,trainLabels,testLabels, classifierSettings):
        classifier= self.trainClassifier(trainData,trainLabels,classifierSettings)
        density = self.findMarginDensity(trainData,classifier)
        score= classifier.score(testData,testLabels)
        #print(score)
        return density, score

        #checks if drift occurs
    def checkDriftAccuracy(self,batch,labels):
        if batch.size==labels.size:
            score=self.classifier.score(batch,labels)
            if self.avAc-score>self.stdDensThresh*self.stdAc:
                return True
            else:
                return False

        #detects the MD3 drift
    def calculateMD3drift(self,batch):
        batchMD= self.findMarginDensity(batch,self.classifier)
        #print("this is selfdensity" + str(self.density))
        #print("this is batchDensity" + str(batchMD))
        #print("this is selfstandardDeviation"+ str(self.stdDens))
        if np.abs(batchMD-self.density)>self.stdDensThresh*self.stdDens:
            return True
        else:
            return False
    def calculateAccuracy(self,data,labels):
        score = self.classifier.score(data,labels)
        #print("this is accuracy"+ str(score))
        return score
    
    @abstractmethod
    def trainClassifier(self,trainData,trainLabels,classifierSettings):
        pass
    @abstractmethod
    def findMarginDensity(self,classifier,trainData):
        pass
    @abstractmethod
    def train(self,trainData,trainLabels):
        pass
    # @abstractmethod
    # def detectDrift(self,batch):
    #     pass

#initiializes drift detector
#python allows no arguement overloading.
class DriftDetector(MDDriftDetector):
    def __init__(self, kernel,C,K,toobig=False):
        

        self.kernel=kernel
        self.C=C
        self.K=K
        self.avAc=None
        self.stdAc=None
        self.stdDens=None
        self.classifier=None
        self.density = None
        self.stdDensThresh=2
        #The ones needed for md3:1
        self.max=self.density
        self.min=self.density
        self.thresh= None

        self.toobig=toobig
        
#needs to be adapted for window stuff.
    def findMarginDensity(self,batch,svm):
        marginDensity=0
        for vect in batch:
            if np.abs(svm.decision_function([vect]))<=1:
                marginDensity= marginDensity+1
        return marginDensity/batch.size
    
    #creates a new classifier then fit it. Return that new Classifier
    #could use wargs for key value pair.

    def trainClassifier(self,data,labels, classifierSettings):
        clf=None
        if self.toobig==True:
            clf = svm.LinearSVC()
            feature_map_nystroem = Nystroem(gamma=.2,
                                random_state=1,
                                n_components=300)
            data_transformed = feature_map_nystroem.fit_transform(data)
        else:
            clf= svm.SVC(kernel=classifierSettings[0],C=classifierSettings[1])
        clf.fit(data,labels)
        return clf

    #multiclass? linear? etc. retrain once found problem.
    def retrain(self,data,labels,kernel,C,K):
        self.classifier.fit(data,labels)
        density = self.findMarginDensity(data)
        self.max=density
        self.min=density
        avAc,stdAc,stdDens= self.KXVal(data,labels,kernel,C,K)
        self.density=density
        self.avAc=avAc
        self.stdAc=stdAc
        self.stdDens=stdDens
    
    def train(self,data,labels):
        #If we pass stuff yea.
        avAc,stdAc,avDens,stdDens= self.KXVal(data,labels,self.K,self.kernel,self.C)
        self.avAc=avAc
        self.stdAc=stdAc
        self.stdDens=stdDens
        classifier= self.trainClassifier(data,labels,(self.kernel,self.C))
        self.classifier= classifier
        self.density = avDens
        self.stdDensThresh=3
##cross avlidaton, gets the distribution components.
    

class MD3V1(MDDriftDetector):
    def __init__(self,kernel,C,threshold,toobig=False):
        classifier= None
        self.kernel=kernel
        self.C=C
        self.max=None
        self.min=None
        self.threshold = threshold
        self.thresh= None
        self.toobig=toobig
        
    def findMarginDensity(self,batch,svm):
        marginDensity=0
        for vect in batch:
            if np.abs(svm.decision_function([vect]))<=1:
                marginDensity= marginDensity+1
        return marginDensity/batch.size
    
    def trainClassifier(self,data,labels, classifierSettings):
        if self.toobig==True:
            clf= svm.LinearSVC(C=self.C)
            feature_map_nystroem = Nystroem(gamma=.2,
                                random_state=1,
                                n_components=300)
            data_transformed = feature_map_nystroem.fit_transform(data)
        else:
            clf= svm.SVC(kernel=self.kernel,C=self.C)
        clf.fit(data,labels)
        return clf
    
    def train(self,data,labels):
        classifier= self.trainClassifier(data,labels,(self.kernel,self.C))
        self.classifier=classifier
        density=self.findMarginDensity(data,classifier)
        self.density=density
        self.max= density
        self.min= density
        self.thresh= self.density * self.threshold

        #threshold=0.075

    #1st version md3
    def calculateMD3drift(self,batch):
        md2=self.findMarginDensity(batch,self.classifier)
        #print(md2)
        if md2>self.max:
            self.max=md2
        if md2<self.min:
            self.min= md2
        if (self.max-self.min)>self.thresh:
            return True
        else: 
            return False



#this is for batch, not sliding window. sliding window forgetting factor sus
#here supposed to query an oracle callback function.

    

            
    # very inefficient, right now, creates new svm. need change the svm, so need to use class svm and fit.
    

class EnsembleMDDriftDetector(MDDriftDetector):
        
    def __init__(self,K,estimator,numberOf,subspaces,bagging):
        #scikit learn parameters
        self.K=K
        self.estimator=estimator
        self.numberOf=numberOf
        self.subspaces=subspaces
        self.bagging=bagging
        #nona ble
        self.avAc=None
        self.stdAc=None
        self.stdDens=None
        self.classifier = None
        self.density= None
        self.stdDensThresh=3


    def trainClassifier(self,data,labels,classifierSettings):
        bagging = BaggingClassifier(classifierSettings[0],n_estimators=classifierSettings[1],max_samples=float(1)
        ,max_features=classifierSettings[2],bootstrap= classifierSettings[3],n_jobs=1)
        bagging.fit(data,labels)
        return bagging


    def findMarginDensity(self,batch,classifier):
        marginDensity=0
        probabilitiesEach= classifier.predict_proba(batch)
        for vect in probabilitiesEach:
            if np.abs(vect[0]-(1-vect[0]))<.5:
                print(np.abs(vect[0]-(1-vect[0])))
                marginDensity= marginDensity+1
        return marginDensity/batch.size
    
    def train(self,data,labels):
        avAc,stdAc,avDens,stdDens= self.KXVal(data,labels,self.K,self.estimator,self.numberOf,self.subspaces,self.bagging)
        self.avAc=avAc
        self.stdAc=stdAc
        self.stdDens=stdDens
        self.classifier = self.trainClassifier(data,labels,(self.estimator,self.numberOf,self.subspaces,self.bagging))
        self.density= avDens
        self.stdDensThresh=2
        


class FuzzyDetector(EnsembleMDDriftDetector):
    
    def membershipFun(self,x):
        membership=0
        confidence = np.abs(x-(1-x))
        if confidence <= .5:
             membership = (np.cos(np.pi*confidence)+1)/2
        return membership

    def findMarginDensity(self,batch,classifier):
        probabilitiesEach = classifier.predict_proba(batch)
        fmd=0
        for vect in probabilitiesEach:
            membership = self.membershipFun(vect[0])
            fmd=fmd+ membership
        return fmd/batch.size


    
dd= DriftDetector("linear",1,2)
dd.train(X,Y)
if dd.calculateMD3drift(X):
    print("yes")
else:
    print("no")

def createBestKNNDetector(data,labels,maxK):
    k_range = range(4, maxK)

    k_scores = np.zeros(maxK)
    X_train, X_test, y_train, y_test = train_test_split(data, labels, train_size=0.30, shuffle=False)


    for k in k_range:
        model = KNeighborsClassifier(n_neighbors=k)
        k_scores[k]= cross_val_score(model, data, labels, cv=10).mean()
        #print(k_scores[k])

    optimizedNeighbors= k_scores.argmax()
    print("optimized neighbors for this is: " + str(optimizedNeighbors))
    return KNeighborsClassifier(n_neighbors=optimizedNeighbors)


no


In [12]:
#from category_encoders import TargetEncoder


realWorldSources= np.array([
    "electricity_dataset.csv",
    "weather_dataset.csv",
    "airline_dataset.csv",
    "spam_dataset.csv"])
#preprocessing.
#LabelEncoder
#iloc and sclice...
#functions to help make arf to usable data
def getCatColumns(pandaFrame):
    names = pandaFrame.select_dtypes(exclude=np.number).columns.tolist()
    return names
    
    #very inefficient, find way to get the last part and see if case, and match 
    #this is there to check what type of file it is and return a pandas frame.
def dfFromUnknownType(link):
    x = re.findall("arff$", link)

    if x:
        data = arff.loadarff(link)[0]
        return pd.DataFrame(data)
    else:
        x = re.findall("csv$",link)
        if x:
            return pd.read_csv(link)
        else:
            print("not a valid File Type, change code or filetype")

def divideIntoBatchesOfX(array,X):
    if array.ndim>1:
        return np.array_split(array,ceil(array.shape[0]/X))
    else:
        return np.array_split(array,ceil(array.size/X))

def getTrainSet(link,data,labels):
    if link== "electricity_dataset.csv":
            A=np.delete(data[:15104],np.array([0,1,7]),1)
            return A,labels[:15104]
            #divideIntoBatchesOfX(data[:15104],365), divideIntoBatchesOfX(labels[:15104],365)
    elif link=="weather_dataset.csv":
        return data[:6053],labels[:6053]


    elif link=="airline_dataset.csv":
        return data[:179794],labels[:179794]


    else:
        return data[:1468],labels[:1468]



def getTestBatches(link, data, labels):
    if link== "electricity_dataset.csv":
        A=np.delete(data[-30208:],np.array([0,1,7]),1)
        return A, labels[-30208:]
        
    elif link=="weather_dataset.csv":
        return data[-12106:],labels[-12106:]

    elif link=="airline_dataset.csv":
        return data[-359589:],labels[-359589:]

    else:
        return data[-2937:],labels[-2937:]
        

    


def fromFiletoData(link,className,encoder):
    df= dfFromUnknownType(link)
    #encode labels
    le = preprocessing.LabelEncoder()
    labels= le.fit_transform(df[[className]].values.ravel())
    df=df.drop(columns=[className],axis=1)
    print("preprocessed",flush=True)
    #encode categorical
    columns = getCatColumns(df)
    if encoder=="Ordinal":
        ord= preprocessing.OrdinalEncoder()
        df[columns[:]]=ord.fit_transform(df[columns[:]])
    elif "OH": 
        df= pd.get_dummies(df)

    elif "Target":
        te = TargetEncoder(cols=columns, smoothing=0, return_df=False)

    #df = te.fit_transform(df, labels)
    scaler= MinMaxScaler()
    print("get the preprocessed things",flush=True)
    df[df.columns] = scaler.fit_transform(df[df.columns])
    #df= scaler.fit_transform(df)
    print(df,flush=True)
    if not encoder=="Target":
        df=df.to_numpy()
    print("tonumpy.",flush=True)
    return df , labels

# link = "agraw1_1_abrupt_drift_0_noise_balanced.arff"
# data,labels = fromArftoData("agraw1_1_abrupt_drift_0_noise_balanced.arff")
# drifDet= DriftDetector(data[:30000],labels[:30000],"linear",1,10)
#print(data)
#print(labels)

In [13]:
#Finding Real World Drift

realWorldSources= np.array([
    "electricity_dataset.csv",
    "weather_dataset.csv",
    "airline_dataset.csv",
    "spam_dataset.csv"])

# as the ammount of data goes to infinity, it will describe the thing the best. so the standard deviation will be... 
# statisticall, sampling a statistic stdDev
#This if for the drift calculations

def getClassifierAccuracy(link,data,labels,classifier):
    numberOfCrossValidations= 40
    trainBatches, totalTrainLabels = getTrainSet(link,data,labels)
    divBatch=divideIntoBatchesOfX(trainBatches,365)
    divLabels=divideIntoBatchesOfX(totalTrainLabels,365)
    accuracies = np.zeros(numberOfCrossValidations)
    for i in range(numberOfCrossValidations):
        rand=random.randint(0,len(divBatch)-1)
        trainData = np.concatenate(divBatch[:rand] + divBatch[rand+1:])
        trainLabels= np.concatenate(divLabels[:rand] + divLabels[rand+1:])
        classifier.fit(trainData,trainLabels)
        accuracies[i]=classifier.score(divBatch[rand],divLabels[rand])
    averageAccuracy= np.average(accuracies)
    standardDevAccuracy= np.std(accuracies)
    return averageAccuracy,standardDevAccuracy

## This way to find Real World drift is deprecated, see common repository.

#Outputs graph of each batch accuracy for each classifier
# def findRealWorldDrift(theSources):
    
#     ##this is getting the accuracies
#     for ind in range(theSources.size):
#         data,labels = fromFiletoData(theSources[ind])
#         trainBatch,trainLabels= getTrainSet(theSources[ind],data,labels)
#                 ##classifiers to try stuff out.
        
#         classifiers =np.array( [
#         createBestKNNDetector(trainBatch,trainLabels,50),
#         svm.SVC(kernel="linear", C=1),
#         svm.SVC(kernel="poly", C=1),
#         svm.SVC(gamma=2, C=1),
#         #GaussianProcessClassifier(1.0 * RBF(1.0),  n_jobs = -1),
#         DecisionTreeClassifier(max_depth=8),
#         RandomForestClassifier(max_depth=8, n_estimators=11, max_features=2),
#         MLPClassifier(alpha=1, max_iter=1000),
#         AdaBoostClassifier(),
#         GaussianNB(),
#         QuadraticDiscriminantAnalysis(),
#             ])
#         ############
#         accuracies= np.zeros(classifiers.size)
#         deviations= np.zeros(classifiers.size)
#         ###finding expected accuracy and deviation, and fitting
#         for classi in range(classifiers.size):
#             accuracies[classi],deviations[classi] = getClassifierAccuracy(theSources[ind],data,labels,classifiers[classi])
#             classifiers[classi].fit(trainBatch,trainLabels)
            
#         ##This is finding when accuracies deviate
#             testBatch,labelBatch= getTestBatches(theSources[ind],data,labels)
#             batchAccuracies=np.zeros(ceil(len(testBatch)))
#             drifts= np.zeros(len(testBatch))
#             for batchNum in range(ceil(len(testBatch))):
#                 batchAccuracy = classifiers[classi].score(testBatch[batchNum],labelBatch[batchNum])
#                 batchAccuracies[batchNum]=batchAccuracy
#                 if accuracies[classi]-1*deviations[classi] > batchAccuracy:
#                     drifts[batchNum]=True

#             print(drifts)
#             path = str(classi) + "path.csv"
#             pd.DataFrame(drifts).to_csv(path)
#             print(accuracies[classi])
#             print(deviations[classi])
#             plt.bar(np.arange(0,len(testBatch)),batchAccuracies)
#             plt.axhline(accuracies[classi]-1*deviations[classi])
#             plt.show()
    
# findRealWorldDrift(realWorldSources)

In [14]:
#Post processing functions

#Metrics:

def getFalsePositive(detections,batchNumber):
    print(detections)
    falsePositives=0
    for i in range(batchNumber-1):
        if(detections[i]):
            falsePositives = falsePositives+1
    return falsePositives/(batchNumber-1)


def getLatency(detections,batchNumber):
    latency=0
    for i in range(batchNumber,detections.size):
        if(not detections[i]):
            latency= latency +1
        else:
            break
    return latency/(detections.size-batchNumber)

    

In [15]:
#Making Nice plot Graphs
def plotNiceLineGraph(data,title,titleX,titleY,labelData):
    if len(data)>1:
        for ind in range(len(data)):
            plt.plot(data[ind],label = labelData[ind])
    else:
        plt.plot(data[0])

    plt.title(title)
    plt.xlabel(titleX, fontsize=15)
    plt.ylabel(titleY, fontsize=15)
    plt.show()

#Bar graph
def plotNiceBarGraph(content,title):
    df = pd.DataFrame(content)
    df.plot(kind='bar',title = title)

#Not 


In [16]:
#Making nice Tables

def tableToLatex(content,columNames,rowNames,fileToSave):
    df = pd.DataFrame(content, columNames, rowNames)
    print(df)
    string = df.to_latex()
    print(string)
    text_file = open(fileToSave, "wt")
    n = text_file.write(string)
    text_file.close()

def tableToExcel(content,columNames,rowNames,fileToSave):
    df = pd.DataFrame(content, columNames, rowNames)
    df.to_excel(fileToSave)
    # print(df)
    # string = df.to_latex()
    # print(string)
    # text_file = open(fileToSave, "wt")
    # n = text_file.write(string)
    # text_file.close()

#tableToLatex(np.array([[.5,22,33],[44,55,66]]),np.array(["a","b"]),np.array(["col1","col2","col3"]),"test.txt")

#To Latex



In [17]:

#Metrics:

def getFalsePositive(detections,batchNumber):
    print(detections)
    falsePositives=0
    for i in range(batchNumber-1):
        if(detections[i]):
            falsePositives = falsePositives+1
    return falsePositives/(batchNumber-1)


def getLatency(detections,batchNumber):
    latency=0
    for i in range(batchNumber,detections.size):
        if(not detections[i]):
            latency= latency +1
        else:
            break
    return latency/(detections.size-batchNumber)

    

In [18]:
#"sea_1_abrupt_drift_0_noise_balanced.arff",
syntheticSources = np.array([
    "sea_1_abrupt_drift_0_noise_balanced.arff",
    "agraw1_1_abrupt_drift_0_noise_balanced.arff",
"agraw2_1_abrupt_drift_0_noise_balanced.arff",
"agraw1_1_gradual_drift_0_noise_balanced_05.arff",
"agraw1_1_gradual_drift_0_noise_balanced_1.arff",
"agraw1_1_gradual_drift_0_noise_balanced_5.arff",
"agraw1_1_gradual_drift_0_noise_balanced_10.arff",
"agraw1_1_gradual_drift_0_noise_balanced_20.arff",
"agraw2_1_gradual_drift_0_noise_balanced_05.arff",
"agraw2_1_gradual_drift_0_noise_balanced_1.arff",
"agraw2_1_gradual_drift_0_noise_balanced_5.arff",
"agraw2_1_gradual_drift_0_noise_balanced_10.arff",
"agraw2_1_gradual_drift_0_noise_balanced_20.arff",
"sea_1_gradual_drift_0_noise_balanced_05.arff",
"sea_1_gradual_drift_0_noise_balanced_1.arff",
"sea_1_gradual_drift_0_noise_balanced_5.arff",
"sea_1_gradual_drift_0_noise_balanced_10.arff",
"sea_1_gradual_drift_0_noise_balanced_20.arff"])

sourceNames=["sea A","agraw1 A","agraw2 A","agraw1_05","agraw1_1","agraw1_5",
"agraw1_10","agraw1_20","agraw2_05","agraw2_1","agraw2_5","agraw2_10","agraw2_20",
"sea_05","sea_1","sea_5","sea_10","sea_20"]

detectorNames=["MD3V1","MD3V2","MD3_Tree","Fuzzy_Tree"]

# for ind in range(2,theSources.size):
#     data,labels = fromArftoData(theSources[ind])
#     #print(data)
#     drifDet= DriftDetector(data[:30000],labels[:30000],"poly",1,5)
#     print("This is the average accuracy of this model:"+str(drifDet.avAc),flush=True)
#     batches = np.array_split(data,10)
#     batches = np.asarray(batches)
#     print("Source:" + theSources[ind])
#     for i in range(3,batches.shape[0]):
#         if drifDet.calculateMD3drift(batches[i]):
#             print("yes")
#         else:
#             print("no")

#using target instead.

#BaggingClassifier(estimator= kwargs[0],n_estimators=kwargs[1],max_samples=float(1),max_features=kwargs[2],bootstrap= kwargs[3],n_jobs=-1)
#





def testOnSource(source,detectors):
    data, labels = fromFiletoData(source)
    latencies= np.zeros(detectors.size)
    fprs= np.zeros(detectors.size)

    
#5,etc,...
#will give false positive rates, and will plot 
def testOnSynthetic(theSources,detectorNum,detectorNames):
    #data of table.
    fprs=np.zeros(shape=(detectorNum, theSources.size))
    latencies=np.zeros(shape=(detectorNum, theSources.size))
    
    for ind in range(theSources.size):
        data,labels = fromFiletoData(theSources[ind],"class","Ordinal")
        trainData = data[:30000]
        trainLabels= labels[:30000]
        
        #this is there to make the detectors KN crashes for me. internal error.
        md3V1 = MD3V1("linear",1,.05)
        drifDet= DriftDetector("linear",1,5)
        # ensembleTreeDetector = EnsembleMDDriftDetector(5,tree.DecisionTreeClassifier(),20,trunc(data.shape[1]/2),False) #needs to be changed...
        # knEnsemble= EnsembleMDDriftDetector(5,KNeighborsClassifier(33),20,trunc(data.shape[1]/2),False)
        # fuzzyTree = FuzzyDetector(5,tree.DecisionTreeClassifier(),20,trunc(data.shape[1]/2),False)
        # fuzzyKn = FuzzyDetector(5,KNeighborsClassifier(33),20,trunc(data.shape[1]/2),False)

        testBatches = divideIntoBatchesOfX(data[-70000:],10000)
        testlabels = divideIntoBatchesOfX(labels[-70000:],10000)
        
        detectors = np.array([md3V1,drifDet])
        for detector in detectors:
            print("next detector", flush=True)
            detector.train(trainData,trainLabels)

        detections = np.zeros(shape=(detectors.size, len(testBatches)))
        marginDensities = np.zeros(shape=(detectors.size, len(testBatches)))
        accuracies = np.zeros(shape=(detectors.size, len(testBatches)))

        print("Source:" + theSources[ind])
        for x in range(detectors.size):
            for i in range(len(testBatches)):
                detections[x][i] =detectors[x].calculateMD3drift(testBatches[i])
                marginDensities[x][i] = detectors[x].findMarginDensity(testBatches[i],detectors[x].classifier)
                accuracies[x][i] = detectors[x].calculateAccuracy(testBatches[i],testlabels[i])
            print(detectors[x].density,detectors[x].stdDens,flush=True)
            plotNiceLineGraph([accuracies[x, :]],detectorNames[x],"Batch","percentage",["Accuracy"])
            plotNiceLineGraph([marginDensities[x, :]],detectorNames[x],"Batch","percentage",["Margin Density"])

        for x in range(detectors.size):
            fprs[x][ind]= getFalsePositive(detections[x],2)
            latencies[x][ind]= getLatency(detections[x],2)
            print(fprs[x][ind])
            print(latencies[x][ind])
    return(fprs,latencies)



fpr,latency= testOnSynthetic(syntheticSources,4,["MD3V2","MD3_Tree","Fuzzy_Tree"])
print(fpr,latency)
#tableToLatex(fpr,detectorNames,sourceNames,"fprRealWorld.txt")
# plotNiceBarGraph(fpr,sourceNames,detectorNames,"False Positive Rate Synthetic")
# plotNiceBarGraph(latency,sourceNames,detectorNames,"Latency Synthetic")
##to change perhaps: expected density etc.


preprocessed
get the preprocessed things
        attrib1   attrib2   attrib3
0      0.730893  0.410086  0.207707
1      0.583365  0.042295  0.761678
2      0.139764  0.694960  0.805231
3      0.275034  0.075385  0.610591
4      0.204916  0.623374  0.184699
...         ...       ...       ...
99995  0.463652  0.663948  0.006662
99996  0.425207  0.335127  0.565219
99997  0.413148  0.637183  0.312550
99998  0.140423  0.439256  0.929862
99999  0.723190  0.877063  0.392545

[100000 rows x 3 columns]
tonumpy.
next detector
next detector
Source:sea_1_abrupt_drift_0_noise_balanced.arff


KeyboardInterrupt: 

In [ ]:
#both 1 etc

# [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
#  [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1.]
#  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
#  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
# [[1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  0.2 1. ]
#  [1.  1.  0.  1.  0.  1.  1.  1.  1.  1.  1.  1.  0.  0.  1.  1.  1.  0. ]
#  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
#  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]]

#Did not get better with higher Ns

# print(fpr)
# print(latency)
#metrics are trash need diff ones
#RealWorldMetrics
def falseAndMiss(detected,real):
    drifts = 0
    fpr=0
    accuracy=0
    missed =0
    for ind in range(real.size):
        if real[ind]==1:
            drifts=drifts+1
            if (detected[ind]<real[ind]):
                missed=missed+1
            else:
                accuracy=accuracy+1
        elif (detected[ind]>real[ind]):
            fpr = fpr+1
    return fpr/(real.size-drifts),missed/drifts,accuracy/drifts
        

def findDifference(detected,real):
    diff=0
    for ind in range(detected.size):
        if not detected[ind]==real[ind]:
            diff=diff + 1
    return diff/detected.size


In [ ]:

#"sea_1_abrupt_drift_0_noise_balanced.arff",

from numpy import genfromtxt


realWorldDrift =np.array([])

sourceNames=["sea A","agraw1 A","agraw2 A","agraw1_05","agraw1_1","agraw1_5",
"agraw1_10","agraw1_20","agraw2_05","agraw2_1","agraw2_5","agraw2_10","agraw2_20",
"sea_05","sea_1","sea_5","sea_10","sea_20"]

detectorNames=["MD3V1","MD3V2","MD3_Tree","MD3KNN","Fuzzy_Tree","FuzzyKNN"]

realWorldSources= np.array([
    "electricity_dataset.csv",
    "weather_dataset.csv",
    "airline_dataset.csv",
    "spam_dataset.csv"])


# for ind in range(2,theSources.size):
#     data,labels = fromArftoData(theSources[ind])
#     #print(data)
#     drifDet= DriftDetector(data[:30000],labels[:30000],"poly",1,5)
#     print("This is the average accuracy of this model:"+str(drifDet.avAc),flush=True)
#     batches = np.array_split(data,10)
#     batches = np.asarray(batches)
#     print("Source:" + theSources[ind])
#     for i in range(3,batches.shape[0]):
#         if drifDet.calculateMD3drift(batches[i]):
#             print("yes")
#         else:
#             print("no")

#using target instead.

#BaggingClassifier(estimator= kwargs[0],n_estimators=kwargs[1],max_samples=float(1),max_features=kwargs[2],bootstrap= kwargs[3],n_jobs=-1)
#




    

def realWorld(link,realDrifts,batchSize,className,detectorNames,tooBig=False):

    data, labels = fromFiletoData(link,className,"Ordinal")
    print("swag",flush=True)

    trainData,trainLabels = getTrainSet(link,data,labels)

    detectors=np.array([
   # MD3V1("linear",1,.075,toobig=tooBig), 
    DriftDetector("linear",1,5,toobig=tooBig),
    EnsembleMDDriftDetector(5,tree.DecisionTreeClassifier(),20,trunc(data.shape[1]/2),False),
    EnsembleMDDriftDetector(5,createBestKNNDetector(trainData,trainLabels,40),20,trunc(data.shape[1]/2),False),
    FuzzyDetector(5,tree.DecisionTreeClassifier(),20,trunc(data.shape[1]/2),False),
    FuzzyDetector(5,createBestKNNDetector(trainData,trainLabels,40),20,trunc(data.shape[1]/2),False)
    ])
    for detector in detectors:
            print("next detector", flush=True)
            detector.train(trainData,trainLabels)

    testData,testLabels = getTestBatches(link,data,labels)
    testBatches = divideIntoBatchesOfX(testData,batchSize)
    testlabels = divideIntoBatchesOfX(testLabels,batchSize)

    print(len(testBatches))
    print(realDrifts.size)

    detections = np.zeros(shape=(detectors.size, len(testBatches)))
    marginDensities = np.zeros(shape=(detectors.size, len(testBatches)))
    accuracies = np.zeros(shape=(detectors.size, len(testBatches)))
    print("Source:" + link)
    fprs= np.zeros(detectors.size)
    misseds= np.zeros(detectors.size)
    driftAccs= np.zeros(detectors.size)
    for x in range(detectors.size):
        for i in range(len(testBatches)):
            detections[x][i] =detectors[x].calculateMD3drift(testBatches[i])
            marginDensities[x][i] = detectors[x].findMarginDensity(testBatches[i],detectors[x].classifier)
            accuracies[x][i] = detectors[x].calculateAccuracy(testBatches[i],testlabels[i])
        plt.bar(np.arange(0,len(testBatches)),accuracies[x, :])
        plt.title(detectorNames[x]+" Accuracy")
        plt.xlabel("Batch")
        plt.axhline(detectors[x].avAc-(detectors[x].stdAc*2))
        plt.show()
        plt.bar(np.arange(0,len(testBatches)),marginDensities[x, :])
        plt.title(detectorNames[x]+" MD")
        plt.xlabel("Batch")
        plt.axhline(detectors[x].density-(detectors[x].stdDens*2))
        plt.show()
        print(detectors[x].stdDens*2)
        fprs[x] , misseds[x] ,driftAccs[x]= falseAndMiss(detections[x, :],realDrifts)

    return fprs, misseds,driftAccs
    

def readDriftToData(fileName):
    df= dfFromUnknownType(fileName)
    df2 = df.iloc[: , 1:]
    myData=df.to_numpy()
    mydata=myData.T
    onlydata=mydata[1, :]
    return onlydata
    
    
    #365 is elec

print(readDriftToData("ElecDatasetPredict.csv"))

def testElec(detectorNames):
    elecDrifts=readDriftToData("ElecDatasetPredict.csv")
    fpr,missed,accuracy =realWorld("electricity_dataset.csv",elecDrifts,365,"label",detectorNames)
    return fpr,missed, accuracy
    

def testSpam(realDrifts,detectorNames):
    batchLengths = [20,50,100]
    fprs = []
    misseds = []
    accuracies = []
    for ind in range(len(realDrifts)):
        fpr,missed,accuracy = realWorld("spam_dataset.csv",realDrifts[ind],batchLengths[ind],"ACTUAL_LABEL",detectorNames)
        fprs= fprs + fpr
        misseds= misseds + missed
        accuracies= accuracies + accuracy
    return fprs, misseds,accuracies

#All encoders saythe same thing. ordinal is okay.
def testAirplane(detectorNames, tooBig=True):
    #drifts in batch
    realAirplaneDrifts = np.zeros(22)
    realAirplaneDrifts[0]=1
    fpr,missed,accuracy = realWorld("airline_dataset.csv",realAirplaneDrifts,17000 ,"Delay",detectorNames,tooBig=True)
    return fpr, missed, accuracy





def testWeather(detectorNames):
    weatherMonthly= np.array([False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,True,True,True,True,True,True,False,False,False,False,True,False,True,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,True,False,False,False,True,True,False,False,False,True,True,False,True,False,False,True,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,False,
    False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,True,True,False,False,True,False,False,False,False,False,False,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,False,False,True,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,True,False,False,True,False,True,False,False,False,False,False,False,False,True,False,True,True,False,False,False])
    weatherMonthly=weatherMonthly.astype(int)
    weatherYearly= np.array([False,True,True,True,False,True,False,False,False,False,True,True,False,False,True,True,True,True,False,False,False,True,False,True,True,False,True,True,True,False,False,True,True])
    weatherYearly=weatherYearly.astype(int)
    #For a month
    fprMonth,missedMonth,accuracy1=realWorld("weather_dataset.csv",weatherMonthly,30,"Label_Rain",detectorNames)
    # tableToLatex(fprMonth,["Month"],detectorNames,"FPR Weather Month.txt")
    # tableToLatex(missedMonth,["Month"],detectorNames,"Missed Weather Month.txt")
    #For a year
    fprYear,missedYear,accuracy2= realWorld("weather_dataset.csv",weatherYearly,365,"Label_Rain",detectorNames)
    fprs=[fprMonth,fprYear]
    missed=[missedMonth,missedYear]
    accuracies=[accuracy1,accuracy2]
    # tableToLatex(fprYear,["Year"],detectorNames,"FPR Weather Year.txt")
    # tableToLatex(missedYear,["Year"],detectorNames,"Missed Weather Year.txt")
    return fprs,missed,accuracies

#print(testWeather())




#def testAirplanes():

#17000 



[0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 0. 1. 0. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 0.
 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1.]


In [ ]:
fprWeather, missWeather, accWeather= testWeather(("MD3V1","MD3V2","MD3_Tree","MD3_KN","Fuzzy_Tree","Fuzzy_K"))


preprocessed
get the preprocessed things
       Unnamed: 0      TEMP      DEWP       SLP     VISIB      WDSP     MXSPD  \
0        0.000000  0.349328  0.454985  0.003725  0.068548  0.315412  0.010120   
1        0.000055  0.367562  0.443369  0.003936  0.217742  0.422939  0.016232   
2        0.000110  0.342610  0.408519  0.004257  0.370968  0.283154  0.012124   
3        0.000165  0.351248  0.415295  0.003770  0.423387  0.236559  0.020140   
4        0.000220  0.322457  0.367861  0.004135  0.391129  0.372760  0.027054   
...           ...       ...       ...       ...       ...       ...       ...   
18154    0.999780  0.486564  0.465634  0.002616  0.798387  0.161290  0.014028   
18155    0.999835  0.369482  0.405615  0.003115  0.633065  0.369176  0.017936   
18156    0.999890  0.245681  0.279768  0.003260  0.625000  0.372760  0.016032   
18157    0.999945  0.182342  0.176186  0.003758  0.737903  0.491039  0.023046   
18158    1.000000  0.092131  0.098742  0.005011  0.657258  0.164875 

KeyboardInterrupt: 

In [ ]:


# [array([.99699,1.        , 1.        , 0.92192192, 1.        , 0.82882883]), array([.93333,1., 1., 1., 1., 1.])]
# [array([0,0.        , 0.        , 0.05714286, 0.        , 0.17142857]), array([0,0.11111111, 0.        , 0.05555556, 0.        , 0.05555556])]
# [array([1,1.        , 1.        , 0.94285714, 1.        , 0.82857143]), array([1,0.88888889, 1.        , 0.94444444, 1.        , 0.94444444])]

# detectorNames=np.array(["MD3_V1","MD3_V2","MD3_Tree","MD3_KN","Fuzzy_Tree","Fuzzy_KN"])

# fprWeather =np.array([[0.996997,1.,1.,0.86486486, 1.,0.78378378],[0.93333333, 1., 1., 0.8, 1., 0.93333333]])
# #[[0., 0., 0., 0.15714286, 0.,0.17142857], [0., 0.11111111, 0., 0.16666667, 0.,0.]]
# accWeather= np.array([[1., 1., 1., 0.84285714, 1.,0.82857143], [1., 0.88888889, 1., 0.83333333, 1.,1.]])

# print(fprWeather)
# print(accWeather)
tableToLatex(fprWeather,np.array(["Monthly","Yearly"]) ,detectorNames,"FPRWeather.txt")
tableToLatex(accWeather,np.array(["Monthly","Yearly"]) ,detectorNames,"AccWeather.txt")
fprElec,missedElec,accelec= testElec(detectorNames)



NameError: name 'fprWeather' is not defined

In [ ]:
# [1.         0.92592593 1.         0.74074074 1.         0.44444444]
# [0.         0.05357143 0.         0.16071429 0.         0.21428571]
# [1.         0.94642857 1.         0.83928571 1.         0.78571429]

#Elec K=5
elecFPR=np.array([0.96296296 ,0.96296296 ,1.     ,    1.  ,       1.,         1.        ])
#[0.         0.10714286 0.         0.         0.01785714 0.        ]
elecAcc=np.array([1.  ,       0.89285714, 1.     ,    1.  ,       0.98214286 ,1.        ])
#Elec K=17

print(fprElec)
print(missedElec)
print(accelec)
tableToLatex(elecFPR,detectorNames,np.array(["Elec FPR"]),"FPRElec.txt")
tableToLatex(elecAcc,detectorNames,np.array(["Elec Acc"]),"AccElec.txt")
tableToExcel(elecFPR,detectorNames,np.array(["Elec FPR"]),"FPRElec.txt")
tableToExcel(elecFPR,detectorNames,np.array(["Elec FPR"]),"FPRElec.txt")


[1.         0.92592593 1.         0.74074074 1.         0.44444444]
[0.         0.05357143 0.         0.16071429 0.         0.21428571]
[1.         0.94642857 1.         0.83928571 1.         0.78571429]
            Elec FPR
MD3_V1      0.962963
MD3_V2      0.962963
MD3_Tree    1.000000
MD3_KN      1.000000
Fuzzy_Tree  1.000000
Fuzzy_KN    1.000000
\begin{tabular}{lr}
\toprule
{} &  Elec FPR \\
\midrule
MD3\_V1     &  0.962963 \\
MD3\_V2     &  0.962963 \\
MD3\_Tree   &  1.000000 \\
MD3\_KN     &  1.000000 \\
Fuzzy\_Tree &  1.000000 \\
Fuzzy\_KN   &  1.000000 \\
\bottomrule
\end{tabular}

            Elec Acc
MD3_V1      1.000000
MD3_V2      0.892857
MD3_Tree    1.000000
MD3_KN      1.000000
Fuzzy_Tree  0.982143
Fuzzy_KN    1.000000
\begin{tabular}{lr}
\toprule
{} &  Elec Acc \\
\midrule
MD3\_V1     &  1.000000 \\
MD3\_V2     &  0.892857 \\
MD3\_Tree   &  1.000000 \\
MD3\_KN     &  1.000000 \\
Fuzzy\_Tree &  0.982143 \\
Fuzzy\_KN   &  1.000000 \\
\bottomrule
\end{tabular}



C:\Users\bapti\AppData\Local\Temp\ipykernel_22296\467450257.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  string = df.to_latex()
C:\Users\bapti\AppData\Local\Temp\ipykernel_22296\467450257.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  string = df.to_latex()


In [ ]:

spamDrift=[[7, 8, 10, 11, 12, 13, 15, 16, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29],
[11, 14, 15, 16, 17, 19, 20, 21, 22, 23, 25, 27, 29, 30, 31, 32, 33, 34, 35, 37, 40, 41, 42, 44, 45, 46, 47, 48, 50, 51, 52, 53, 55, 56, 57, 58],
[10, 14, 21, 26, 32, 33, 35, 36, 37, 39, 40, 41, 42, 47, 49, 50, 51, 52, 53, 54, 56, 57, 58, 60, 61, 62, 63, 67, 68, 72, 73, 76, 78, 80, 81, 82, 83,
 85, 86, 87, 88, 91, 92, 93, 95, 97, 98, 99, 102, 103, 104, 105, 107, 108, 109, 110, 111, 112, 114, 115, 116, 117, 118, 119, 120, 123, 124, 125, 126, 127, 128, 130, 131, 132, 135, 136, 137, 138, 139, 141, 142, 143, 145]]

spambatlengths=[29,58,146] #146 147
spdrift1 = np.zeros(29)
spdrift2 = np.zeros(58)
spdrift3 = np.zeros(146)
spdriftLengths=[spdrift1,spdrift2,spdrift3]

for x in range(len(spamDrift)):
    for num in spamDrift[x]:
        spdriftLengths[x][num-1]=1

# print(fprElec)
# print(missedElec)
# print(accelec)
fprSpam,missedSpam,spamacc = testSpam(spdriftLengths,detectorNames)

preprocessed
get the preprocessed things


KeyboardInterrupt: 

In [ ]:
# OrdinalEncoder
fprair= [0., 0. ,1. ,1. ,1. ,0.]
# [1. 1. 0. 0. 0. 1.]
accair= [0., 0., 1., 1., 1., 0.]

tableToLatex(fprair,detectorNames,np.array(["Air FPR"]),"FPRAIR.txt")
tableToLatex(accair,detectorNames,np.array(["Air Acc"]),"AccAir.txt")

tableToExcel(fprair,detectorNames,np.array(["Air FPR"]),"FPRAIR.xmnls")
tableToExcel(accair,detectorNames,np.array(["Air Acc"]),"AccAir.xmnls")


#[0.95454545 0.95454545] [0. 0.]
fprair,missedair,accair = testAirplane(detectorNames,tooBig=True)
print(fprair)
print(missedair)
print(accair)

            Air FPR
MD3V1           0.0
MD3V2           0.0
MD3_Tree        1.0
MD3KNN          1.0
Fuzzy_Tree      1.0
FuzzyKNN        0.0
\begin{tabular}{lr}
\toprule
{} &  Air FPR \\
\midrule
MD3V1      &      0.0 \\
MD3V2      &      0.0 \\
MD3\_Tree   &      1.0 \\
MD3KNN     &      1.0 \\
Fuzzy\_Tree &      1.0 \\
FuzzyKNN   &      0.0 \\
\bottomrule
\end{tabular}

            Air Acc
MD3V1           0.0
MD3V2           0.0
MD3_Tree        1.0
MD3KNN          1.0
Fuzzy_Tree      1.0
FuzzyKNN        0.0
\begin{tabular}{lr}
\toprule
{} &  Air Acc \\
\midrule
MD3V1      &      0.0 \\
MD3V2      &      0.0 \\
MD3\_Tree   &      1.0 \\
MD3KNN     &      1.0 \\
Fuzzy\_Tree &      1.0 \\
FuzzyKNN   &      0.0 \\
\bottomrule
\end{tabular}



C:\Users\bapti\AppData\Local\Temp\ipykernel_29496\467450257.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  string = df.to_latex()
C:\Users\bapti\AppData\Local\Temp\ipykernel_29496\467450257.py:6: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  string = df.to_latex()


preprocessed
get the preprocessed things


c:\Users\bapti\anaconda3\envs\env\lib\site-packages\sklearn\preprocessing\_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


        Unnamed: 0    Flight      Time    Length  Airline_b'9E'  \
0         0.000000  0.034302  0.003499  0.312977            0.0   
1         0.000002  0.199283  0.003499  0.338931            0.0   
2         0.000004  0.307052  0.006998  0.251908            0.0   
3         0.000006  0.315500  0.006998  0.297710            0.0   
4         0.000007  0.013695  0.013996  0.308397            0.0   
...            ...       ...       ...       ...            ...   
539378    0.999993  0.022655  1.000000  0.497710            0.0   
539379    0.999994  0.050813  1.000000  0.465649            0.0   
539380    0.999996  0.077819  1.000000  0.389313            0.0   
539381    0.999998  0.009855  1.000000  0.477863            0.0   
539382    1.000000  0.184436  1.000000  0.459542            0.0   

        Airline_b'AA'  Airline_b'AS'  Airline_b'B6'  Airline_b'CO'  \
0                 0.0            0.0            0.0            1.0   
1                 0.0            0.0            0.0    

KeyboardInterrupt: 

In [ ]:
#For


# [[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
#  [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1.]
#  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
#  [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
# [[1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  0.2 1. ]
#  [1.  1.  0.  1.  0.  1.  1.  1.  1.  1.  1.  1.  0.  0.  1.  1.  1.  0. ]
#  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
#  [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]]

sourceNames=np.array(["sea A","agraw1 A","agraw2 A","agraw1_05","agraw1_1","agraw1_5",
"agraw1_10","agraw1_20","agraw2_05","agraw2_1","agraw2_5","agraw2_10","agraw2_20",
"sea_05","sea_1","sea_5","sea_10","sea_20"])

resultsFPR= np.array([[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1],[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],[0 ,0 ,0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0],[1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1],[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1]])
resultsLatency= np.array([[0.2,0.2,1,0.2,.2,.2,.2,.4,1,1,1,1,1,.2,.2,.2,.2,.4],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,.2,.2],[0,0,0,0,0,0,0,.2,0,0,0,0,0,0,0,0,0,0],[1.,1., 1.,1., 1. , 0. , 1. , 1. , 1., 1.,1. , 1. , 0. , 1. , 1.,  1., 0.2 ,1. ],[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0],[1. , 1.,  0. , 1. , 0. , 1. , 1. , 1. , 1. , 1. , 1. , 1. , 0. , 0. , 1. , 1. , 1. , 0. ]])
print(resultsLatency)


fprAbrupt =resultsFPR[:,:3]
latencyAbrupt = resultsLatency[:,:3]

fprGradual = resultsFPR[:,-15:]
latencyGradual=resultsLatency[:,-15:]

fprsea = fprGradual[:,-5:]
fprag1 = fprGradual[:,:5]
fprag2 = fprGradual[:,5:10]
latsea =latencyGradual[:,-5:]
latag1 =latencyGradual[:,:5]
latag2 = latencyGradual[:,5:10]


colsAb=sourceNames[:3]
colsGrad= sourceNames[-15:]
colsGradSea = colsGrad[-5:]
colsGradAg1 = colsGrad[:5]
colsGradAg2 = colsGrad[5:10]

detectorNames=np.array(["MD3_V1","MD3_V2","MD3_Tree","MD3_KN","Fuzzy_Tree","Fuzzy_KN"])


#All of them
# tableToLatex(np.transpose(resultsFPR),sourceNames,detectorNames,"FPRsSynthetic.txt")
# tableToLatex(np.transpose(resultsLatency),sourceNames,detectorNames,"LatencySynthetic.txt")

tableToExcel(np.transpose(resultsFPR),sourceNames,detectorNames,"faslepositivesSynthetic.xlsx")
tableToExcel(np.transpose(resultsLatency),sourceNames,detectorNames,"LatencySSynthetic.xlsx")

# #abrupt
# tableToLatex(fprAbrupt,detectorNames,colsAb,"FPRsSyntheticAbrupt.txt")
# tableToLatex(latencyAbrupt,detectorNames,colsAb,"LatencyAbrupt.txt")

# #Gradual
# tableToLatex(fprsea,detectorNames,colsGradSea,"FRPGradualSea.txt")
# tableToLatex(fprag1,detectorNames,colsGradAg1,"FRPGradualAg1.txt")
# tableToLatex(fprag2,detectorNames,colsGradAg2,"FRPGradualAg2.txt")
# tableToLatex(latsea,detectorNames,colsGradSea,"LatencyGradualSea.txt")
# tableToLatex(latag1,detectorNames,colsGradAg1,"LatencyGradualAg1.txt")
# tableToLatex(latag2,detectorNames,colsGradAg2,"LatencyGradualAg2.txt")
#def tableToLatex(content,columNames,rowNames,fileToSave):


[[0.2 0.2 1.  0.2 0.2 0.2 0.2 0.4 1.  1.  1.  1.  1.  0.2 0.2 0.2 0.2 0.4]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.2 0.2]
 [0.  0.  0.  0.  0.  0.  0.  0.2 0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [1.  1.  1.  1.  1.  0.  1.  1.  1.  1.  1.  1.  0.  1.  1.  1.  0.2 1. ]
 [0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. ]
 [1.  1.  0.  1.  0.  1.  1.  1.  1.  1.  1.  1.  0.  0.  1.  1.  1.  0. ]]
